## NetShark Info

### Imports & Setup

In [ ]:
from datetime import timedelta

from steelscript.netshark.core import NetShark
from steelscript.netshark.core.types import Value, Key
from steelscript.netshark.core.filters import NetSharkFilter, TimeFilter
from steelscript.common import UserAuth

In [ ]:
hostname = "NETSHARK.HOSTNAME.COM"
username = "USERNAME"
password = "PASSWORD"

In [ ]:
netshark = NetShark(hostname, auth=UserAuth(username, password))
netshark

### Source - Capture Job

Pick the first capture job that's running

In [ ]:
jobs = netshark.get_capture_jobs()
source = None
for j in jobs:
    if j.get_status()['state'] == 'RUNNING':
        source = j
        break
source

### Columns

Select the key and value columns to collect.  The set of "Key" column define how the "Value" column data is aggregated.  For example a single Key column of "server_port" as below indicates that a row will be defined for each unique server port seen.

In [ ]:
columns = [
    Key(netshark.columns.tcp.server_port),
    Value(netshark.columns.generic.bytes)
]

### Filters


In [ ]:
timefilter = TimeFilter.parse_range('last 1s')

In [ ]:
filters = [timefilter]

### Create the view

In [ ]:
view = netshark.create_view(source, columns, filters)

In [ ]:
view

In [ ]:
ti = view.get_timeinfo()
ti

Most times from shark are unix epoch in nanoseconds (so nanoseconds since Jan 1, 1970)

In [ ]:
(ti['end'] - ti['start']) / 1000000000

### Retrieve data

In [ ]:
data = view.get_data(aggregated=True)
packets = data[0]['p']
t = data[0]['t']
rows = data[0]['vals']

print('Counted %d packets starting at %s, yielding %d rows' % 
      (packets, str(t), len(rows)))

In [ ]:
data

Each row is a tuple of (*port*, *bytes*) -- the requested key and value columns

In [ ]:
rows[0]

Sort the rows by the bytes column, column 1 in each row:

In [ ]:
rows.sort(lambda a, b: cmp(a[1], b[1]), reverse=True)
rows[:10]

### Analyze the data, looking for a single port

In [ ]:
def find_port(data, port):
    total = 0
    for sample in data:
        for v in sample['vals']:
            if v[0] == port:
                print "%s: %s" % (str(sample['t']), v[1])
                total = total + v[1]
    print "Total: %d" % total

In [ ]:
find_port(data, 80)

In [ ]:
data_1min = view.get_data(aggregated=False, delta=timedelta(seconds=60))
print "Data points: %d" % len(data_1min)
find_port(data_1min, 80)

In [ ]:
data_10sec = view.get_data(aggregated=False, delta=timedelta(seconds=10))
print "Data points: %d" % len(data_10sec)
find_port(data_10sec, 80)